In [1]:
import pandas as pd
import numpy as np

# from google.colab import drive
import os
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import glob

In [2]:
# Se buscan y abren todos los ficheros csv de la carpeta path
path = r'C:\Users\MIA\OneDrive - UNIR\MasterIA\Asignaturas\2ºCuatrimestre\008 PRÁCTICAS EN EMPRESA (ONLINE)\99 Notebooks\datasets\test1'

all_files = glob.glob(path + "/*.csv")
li = []

for filename in all_files:
    print("Se ha leido:  " + filename)
    df = pd.read_csv(filename, index_col=None, header=None)
    li.append(df)
df = pd.concat(li, axis=0, ignore_index=True)

# Se dan nombres a las columnas
df.columns = ['Event_Number','Ini_E[MeV]','Ini_X[cm]','Ini_Y[cm]','Ini_Z[cm]','Ini_Theta','Ini_Phi','Number_of_hits']

Se ha leido:  C:\Users\MIA\OneDrive - UNIR\MasterIA\Asignaturas\2ºCuatrimestre\008 PRÁCTICAS EN EMPRESA (ONLINE)\99 Notebooks\datasets\test1\simple_electrons_at_100MeV.csv
Se ha leido:  C:\Users\MIA\OneDrive - UNIR\MasterIA\Asignaturas\2ºCuatrimestre\008 PRÁCTICAS EN EMPRESA (ONLINE)\99 Notebooks\datasets\test1\simple_electrons_at_10MeV.csv
Se ha leido:  C:\Users\MIA\OneDrive - UNIR\MasterIA\Asignaturas\2ºCuatrimestre\008 PRÁCTICAS EN EMPRESA (ONLINE)\99 Notebooks\datasets\test1\simple_electrons_at_133MeV.csv
Se ha leido:  C:\Users\MIA\OneDrive - UNIR\MasterIA\Asignaturas\2ºCuatrimestre\008 PRÁCTICAS EN EMPRESA (ONLINE)\99 Notebooks\datasets\test1\simple_electrons_at_178MeV.csv
Se ha leido:  C:\Users\MIA\OneDrive - UNIR\MasterIA\Asignaturas\2ºCuatrimestre\008 PRÁCTICAS EN EMPRESA (ONLINE)\99 Notebooks\datasets\test1\simple_electrons_at_56MeV.csv


In [3]:
print(df.columns)
df.shape

Index(['Event_Number', 'Ini_E[MeV]', 'Ini_X[cm]', 'Ini_Y[cm]', 'Ini_Z[cm]',
       'Ini_Theta', 'Ini_Phi', 'Number_of_hits'],
      dtype='object')


(1976, 8)

#### OBJETIVO: Tener un dataframe con esta forma: 'Event_Number','Ini_E[MeV]','Ini_Theta','Ini_Phi','Number_of_hits','Plane_h','X_h[cm]','Y_h[cm]','Z_h[cm]','Time_h[ns]'

In [4]:
dfFirma =df.copy()

# TRATAMIENTO DE DATOS NULOS
# Rellenamos el NaN del number of hits de cada hit con -1, puesto que no tiene sentido que tenga valor alguno. Se hace para poder filtrar facilmente en los bucles for.
dfFirma['Number_of_hits'] = dfFirma['Number_of_hits'].fillna(-1)
dfFirma['Ini_Theta'] = dfFirma['Ini_Theta'].fillna(-1)
dfFirma['Ini_Phi'] = dfFirma['Ini_Phi'].fillna(-1)

# creamos un dataframe vacío con todas las columnas que vamos a incluir (eventos + hits)
dfJoin = pd.DataFrame(columns=['Event_Number','Ini_E[MeV]','Ini_X[cm]','Ini_Y[cm]','Ini_Z[cm]','Ini_Theta','Ini_Phi','Number_of_hits','Plane_h','X_h[cm]','Y_h[cm]','Z_h[cm]','Time_h[ns]'])

for i in range(len(dfFirma)):
    porcentaje = (i * 100) / df.shape[0]
    print(str(i)+'/'+str(df.shape[0])+"\t"+str(round(porcentaje,3))+"%")
    if dfFirma["Number_of_hits"][i] > 0: # Fila de Evento
        # Se recogen los valores del evento: Event_Number, Ini_E[MeV], Ini_Theta, Ini_Phi, Number_of_hits
        event_number =    dfFirma.iloc[i,0]
        initial_energy =  dfFirma.iloc[i,1]
        initial_x =       dfFirma.iloc[i,2]
        initial_y =       dfFirma.iloc[i,3]
        initial_z =       dfFirma.iloc[i,4]
        initial_tetha =   dfFirma.iloc[i,5]
        initial_phi =     dfFirma.iloc[i,6]
        number_of_hits =  dfFirma.iloc[i,7]
        
        # Se corrigen los IDs para no se solapen los ids de eventos de diferentes ficheros
        if initial_energy == 10.0:
            event_number = event_number + 1000
        if initial_energy == 56.0:
            event_number = event_number + 2000
        if initial_energy == 100.0:
            event_number = event_number + 3000
        if initial_energy == 133.0:
            event_number = event_number + 4000
        if initial_energy == 178.0:
            event_number = event_number + 5000
        
    else: # Fila de Hit
        hit_plane = dfFirma.iloc[i,0]
        hit_x =     dfFirma.iloc[i,1]
        hit_y =     dfFirma.iloc[i,2]
        hit_z =     dfFirma.iloc[i,3]
        hit_time =  dfFirma.iloc[i,4]  

        # Se guardan todos los datos en el nuevo dataframe
        dfJoin = dfJoin.append({'Event_Number':event_number,
           'Ini_X[cm]':initial_x,
           'Ini_Y[cm]':initial_y,
           'Ini_Z[cm]':initial_z,
           'Ini_E[MeV]':initial_energy,
           'Ini_Theta':initial_tetha,
           'Ini_Phi':initial_phi,
           'Number_of_hits':number_of_hits,
           'Plane_h':hit_plane,
           'X_h[cm]':hit_x,
           'Y_h[cm]':hit_y,
           'Z_h[cm]':hit_z,
           'Time_h[ns]':hit_time}, 
           ignore_index=True)

# # Cambiamos el signo a la altura para poder representar los planos del detector en altura. 0cm=suelo=4ºplaca
# dfFull["Z_h[cm]"] = dfFull["Z_h[cm]"].apply(lambda x: x*-1)

# # Cambiamos el nombre a la columna de indice. Así cada fila se corresponde con un hit que posee un único ID.
dfJoin.index.name = 'Hit_Id'

0/1976	0.0%
1/1976	0.051%
2/1976	0.101%
3/1976	0.152%
4/1976	0.202%
5/1976	0.253%
6/1976	0.304%
7/1976	0.354%
8/1976	0.405%
9/1976	0.455%
10/1976	0.506%
11/1976	0.557%
12/1976	0.607%
13/1976	0.658%
14/1976	0.709%
15/1976	0.759%
16/1976	0.81%
17/1976	0.86%
18/1976	0.911%
19/1976	0.962%
20/1976	1.012%
21/1976	1.063%
22/1976	1.113%
23/1976	1.164%
24/1976	1.215%
25/1976	1.265%
26/1976	1.316%
27/1976	1.366%
28/1976	1.417%
29/1976	1.468%
30/1976	1.518%
31/1976	1.569%
32/1976	1.619%
33/1976	1.67%
34/1976	1.721%
35/1976	1.771%
36/1976	1.822%
37/1976	1.872%
38/1976	1.923%
39/1976	1.974%
40/1976	2.024%
41/1976	2.075%
42/1976	2.126%
43/1976	2.176%
44/1976	2.227%
45/1976	2.277%
46/1976	2.328%
47/1976	2.379%
48/1976	2.429%
49/1976	2.48%
50/1976	2.53%
51/1976	2.581%
52/1976	2.632%
53/1976	2.682%
54/1976	2.733%
55/1976	2.783%
56/1976	2.834%
57/1976	2.885%
58/1976	2.935%
59/1976	2.986%
60/1976	3.036%
61/1976	3.087%
62/1976	3.138%
63/1976	3.188%
64/1976	3.239%
65/1976	3.289%
66/1976	3.34%
67/1976	3.391

535/1976	27.075%
536/1976	27.126%
537/1976	27.176%
538/1976	27.227%
539/1976	27.277%
540/1976	27.328%
541/1976	27.379%
542/1976	27.429%
543/1976	27.48%
544/1976	27.53%
545/1976	27.581%
546/1976	27.632%
547/1976	27.682%
548/1976	27.733%
549/1976	27.783%
550/1976	27.834%
551/1976	27.885%
552/1976	27.935%
553/1976	27.986%
554/1976	28.036%
555/1976	28.087%
556/1976	28.138%
557/1976	28.188%
558/1976	28.239%
559/1976	28.289%
560/1976	28.34%
561/1976	28.391%
562/1976	28.441%
563/1976	28.492%
564/1976	28.543%
565/1976	28.593%
566/1976	28.644%
567/1976	28.694%
568/1976	28.745%
569/1976	28.796%
570/1976	28.846%
571/1976	28.897%
572/1976	28.947%
573/1976	28.998%
574/1976	29.049%
575/1976	29.099%
576/1976	29.15%
577/1976	29.2%
578/1976	29.251%
579/1976	29.302%
580/1976	29.352%
581/1976	29.403%
582/1976	29.453%
583/1976	29.504%
584/1976	29.555%
585/1976	29.605%
586/1976	29.656%
587/1976	29.706%
588/1976	29.757%
589/1976	29.808%
590/1976	29.858%
591/1976	29.909%
592/1976	29.96%
593/1976	30.01%
594/1

1028/1976	52.024%
1029/1976	52.075%
1030/1976	52.126%
1031/1976	52.176%
1032/1976	52.227%
1033/1976	52.277%
1034/1976	52.328%
1035/1976	52.379%
1036/1976	52.429%
1037/1976	52.48%
1038/1976	52.53%
1039/1976	52.581%
1040/1976	52.632%
1041/1976	52.682%
1042/1976	52.733%
1043/1976	52.783%
1044/1976	52.834%
1045/1976	52.885%
1046/1976	52.935%
1047/1976	52.986%
1048/1976	53.036%
1049/1976	53.087%
1050/1976	53.138%
1051/1976	53.188%
1052/1976	53.239%
1053/1976	53.289%
1054/1976	53.34%
1055/1976	53.391%
1056/1976	53.441%
1057/1976	53.492%
1058/1976	53.543%
1059/1976	53.593%
1060/1976	53.644%
1061/1976	53.694%
1062/1976	53.745%
1063/1976	53.796%
1064/1976	53.846%
1065/1976	53.897%
1066/1976	53.947%
1067/1976	53.998%
1068/1976	54.049%
1069/1976	54.099%
1070/1976	54.15%
1071/1976	54.2%
1072/1976	54.251%
1073/1976	54.302%
1074/1976	54.352%
1075/1976	54.403%
1076/1976	54.453%
1077/1976	54.504%
1078/1976	54.555%
1079/1976	54.605%
1080/1976	54.656%
1081/1976	54.706%
1082/1976	54.757%
1083/1976	54.808

1488/1976	75.304%
1489/1976	75.354%
1490/1976	75.405%
1491/1976	75.455%
1492/1976	75.506%
1493/1976	75.557%
1494/1976	75.607%
1495/1976	75.658%
1496/1976	75.709%
1497/1976	75.759%
1498/1976	75.81%
1499/1976	75.86%
1500/1976	75.911%
1501/1976	75.962%
1502/1976	76.012%
1503/1976	76.063%
1504/1976	76.113%
1505/1976	76.164%
1506/1976	76.215%
1507/1976	76.265%
1508/1976	76.316%
1509/1976	76.366%
1510/1976	76.417%
1511/1976	76.468%
1512/1976	76.518%
1513/1976	76.569%
1514/1976	76.619%
1515/1976	76.67%
1516/1976	76.721%
1517/1976	76.771%
1518/1976	76.822%
1519/1976	76.872%
1520/1976	76.923%
1521/1976	76.974%
1522/1976	77.024%
1523/1976	77.075%
1524/1976	77.126%
1525/1976	77.176%
1526/1976	77.227%
1527/1976	77.277%
1528/1976	77.328%
1529/1976	77.379%
1530/1976	77.429%
1531/1976	77.48%
1532/1976	77.53%
1533/1976	77.581%
1534/1976	77.632%
1535/1976	77.682%
1536/1976	77.733%
1537/1976	77.783%
1538/1976	77.834%
1539/1976	77.885%
1540/1976	77.935%
1541/1976	77.986%
1542/1976	78.036%
1543/1976	78.08

1951/1976	98.735%
1952/1976	98.785%
1953/1976	98.836%
1954/1976	98.887%
1955/1976	98.937%
1956/1976	98.988%
1957/1976	99.038%
1958/1976	99.089%
1959/1976	99.14%
1960/1976	99.19%
1961/1976	99.241%
1962/1976	99.291%
1963/1976	99.342%
1964/1976	99.393%
1965/1976	99.443%
1966/1976	99.494%
1967/1976	99.545%
1968/1976	99.595%
1969/1976	99.646%
1970/1976	99.696%
1971/1976	99.747%
1972/1976	99.798%
1973/1976	99.848%
1974/1976	99.899%
1975/1976	99.949%


In [5]:
# dfJoin.describe()

In [6]:
# dfJoin.head(10)

In [7]:
# TEST DE VARIOS EVENTOS CON MULTIPLES HITS
# dfEventosHits = dfJoin.sort_values(by=['Number_of_hits','Event_Number'],ascending=False).head(97)
dfEventosHits = dfJoin.sort_values(by=['Number_of_hits','Event_Number'],ascending=False)
# dfEventosHits = dfJoin.sort_values(by=['Event_Number'],ascending=True)
# dfEventosHits=dfJoin.copy()

#Convertimos a enteros los id de evento, energía inicial, plano, y numero de hits.
dfEventosHits['Event_Number'] = dfEventosHits['Event_Number'].apply(np.int64)
dfEventosHits['Ini_E[MeV]'] = dfEventosHits['Ini_E[MeV]'].apply(np.int64)
dfEventosHits['Plane_h'] = dfEventosHits['Plane_h'].apply(np.int64)
dfEventosHits['Number_of_hits'] = dfEventosHits['Number_of_hits'].apply(np.int64)

# Guardamos todos los eventos y hits en un csv temporal
dfEventosHits.to_csv("output/test1/dataframes/eventosPreparadosParaGenerarRayos.csv")

dfEventosHits.head(100000)

,Event_Number,Ini_E[MeV],Ini_X[cm],Ini_Y[cm],Ini_Z[cm],Ini_Theta,Ini_Phi,Number_of_hits,Plane_h,X_h[cm],Y_h[cm],Z_h[cm],Time_h[ns]
Hit_Id,,,,,,,,,,,,,
1124,5097,178,1.0,1.0,-220.0,0.0,0.0,9,1,6.3,6.05,-187.3,1.10231
1125,5097,178,1.0,1.0,-220.0,0.0,0.0,9,2,6.3,-6.05,-134.8,3.09049
1126,5097,178,1.0,1.0,-220.0,0.0,0.0,9,3,6.3,-6.05,-96.9,4.22786
1127,5097,178,1.0,1.0,-220.0,0.0,0.0,9,4,-31.5,-6.05,-13.2,8.61922
1128,5097,178,1.0,1.0,-220.0,0.0,0.0,9,2,6.3,6.05,-134.8,2.93518
...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,1005,10,1.0,1.0,-220.0,0.0,0.0,1,1,6.3,6.05,-187.3,11.41100
350,1004,10,1.0,1.0,-220.0,0.0,0.0,1,1,6.3,6.05,-187.3,110.82800
349,1003,10,1.0,1.0,-220.0,0.0,0.0,1,1,6.3,6.05,-187.3,105.42600


In [8]:
import winsound
frequency = 2500  # Set Frequency To 2500 Hertz
duration = 200  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)
winsound.Beep(frequency, duration)
winsound.Beep(frequency, duration)